In [2]:
print('Hello')

Hello


In [ ]:
#Updated 03 Oct 2023
import win32com.client
# import Common
from typing import List, Dict, Any
import os
from time import sleep

ComServer = win32com.client.Dispatch("SQLAcc.BizApp")

def KillApp():
    os.system('cmd /c "taskkill /IM "SQLACC.exe" /F"')
    sleep(2) #sleep 2 sec


def CheckLogin():
    global ComServer
    ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
    B = ComServer.IsLogin
    if B == True:
        ComServer.Logout()
        KillApp()
    ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
    try:    
        ComServer.Login("ADMIN", "ADMIN", #UserName, Password
                        "\\DESKTOP-C8CJ70C\eStream\SQLAccounting\Share\Default.DCF", #DCF file
                        "ACC-0001.FDB") #Database Name
    except Exception as e:
        print("Oops !", e)


def ShowResult(ADataset):
    if ADataset.RecordCount > 0:
        while not ADataset.eof:
            fc = ADataset.Fields.Count
            for x in range(fc):
                fn = ADataset.Fields.Items(x).FieldName
                fv = ADataset.FindField(fn).AsString
                lresult = "Index : "+ str(x) + " FieldName : " + fn + " Value : " + fv
                print (lresult)
            print("====")
            ADataset.Next()
    else:
        print ("Record Not Found")
        
        
def GetResult(ADataset) -> List[Dict[str, Any]] | None:
    Result = []
    if ADataset.RecordCount > 0:
        while not ADataset.eof:
            fc = ADataset.Fields.Count
            lData = {}
            for x in range(fc):
                fn = ADataset.Fields.Items(x).FieldName
                fv = ADataset.FindField(fn).AsString
                lData[fn] = fv
            Result.append(lData)
            ADataset.Next()
        return Result
    else:
        return None
        
def QuotedStr(ACode):
    return "'" + ACode.replace("'", "''") + "'"
	
# login
# CheckLogin()

    

def GetListData():
    lSQL = "SELECT * FROM AGENT "
    lDataSet = ComServer.DBManager.NewDataSet(lSQL)
    return GetResult(lDataSet)


from typing import Literal
def DelData(DocNo: str, Object: Literal['PH_PI', 'SL_IV']):
    #Deleting only work if the record never not knock off by Payment or Credit Note
    BizObject = ComServer.BizObjects.Find(Object)
    
    lDocKey = BizObject.FindKeyByRef("DocNo", DocNo)
        
    if lDocKey is None:
        print ("Record Not Found")
        raise Exception(f"Record Not Found for {DocNo}")
    else:
        BizObject.Params.Find("Dockey").Value = lDocKey
        BizObject.Open()
        BizObject.Delete()        
        print (f"Deleted {DocNo} successfully")
        
# # # Delete from IV-00151 to IV-00201
# for i in range(260, 263):
#     try:
#         DelData(f"PI-{i:05d}", 'PH_PI')
#     except Exception as e:
#         print(f"Error deleting {i:05d}: {e}")
#         # Optional: Log the error or take other actions as needed
        
# for i in range(217, 220):
#     try:
#         DelData(f"IV-{i:05d}", 'SL_IV')
#     except Exception as e:
#         print(f"Error deleting {i:05d}: {e}")
#         # Optional: Log the error or take other actions as needed


# BizObject = ComServer.BizObjects.Find("AP_SUPPLIER")
# # lMain = BizObject.DataSets.Find("MainDataSet")
# # lDetail = BizObject.DataSets.Find("cdsDocDetail")

# lDocKey = BizObject.FindKeyByRef("CODE", "1034")
# if lDocKey is None:
#     print("Record Not Found")
# else:
#     print(lDocKey)
page = 1
limit = 100
lSQL = f"""SELECT DOCNO, CODE, DOCDATE FROM SL_IV 
WHERE EXTRACT(MONTH FROM DOCDATE) = 3 
AND EXTRACT(YEAR FROM DOCDATE) = 2025 
ROWS {(page-1)*limit+1} TO {page*limit}
;
"""
lSQL = f"""SELECT SL.DOCNO, SL.CODE AS SL_CODE, SL.DOCDATE AS SL_DATE,
    PI.DOCNO AS PI_DOCNO, PI.CODE AS PI_CODE, PI.DOCDATE AS PI_DATE,
    PI.DOCAMT AS PI_AMOUNT, SL.DOCAMT AS SL_AMOUNT
FROM SL_IV SL
JOIN PH_PI PI 
    ON SL.DOCDATE = PI.DOCDATE
    AND SUBSTRING(SL.CODE FROM 5) = PI.CODE
WHERE EXTRACT(MONTH FROM SL.DOCDATE) = 3
    AND EXTRACT(YEAR FROM SL.DOCDATE) = 2025
ROWS {(page-1)*limit+1} TO {page*limit};
"""
lDataSet = ComServer.DBManager.NewDataSet(lSQL)
binded = GetResult(lDataSet)

<>:25: SyntaxWarning: invalid escape sequence '\e'
<>:25: SyntaxWarning: invalid escape sequence '\e'
C:\Users\D&B Trading\AppData\Local\Temp\ipykernel_9392\1913084338.py:25: SyntaxWarning: invalid escape sequence '\e'
  "\\DESKTOP-C8CJ70C\eStream\SQLAccounting\Share\Default.DCF", #DCF file


[{'DOCNO': 'IV-00136',
  'SL_CODE': '300-1026',
  'SL_DATE': '14/03/2025',
  'PI_DOCNO': 'PI-00178',
  'PI_CODE': '1026',
  'PI_DATE': '14/03/2025',
  'PI_AMOUNT': '274.5',
  'SL_AMOUNT': '20'},
 {'DOCNO': 'IV-00137',
  'SL_CODE': '300-1034',
  'SL_DATE': '11/03/2025',
  'PI_DOCNO': 'PI-00179',
  'PI_CODE': '1034',
  'PI_DATE': '11/03/2025',
  'PI_AMOUNT': '375.15',
  'SL_AMOUNT': '50'},
 {'DOCNO': 'IV-00138',
  'SL_CODE': '300-1039',
  'SL_DATE': '03/03/2025',
  'PI_DOCNO': 'PI-00180',
  'PI_CODE': '1039',
  'PI_DATE': '03/03/2025',
  'PI_AMOUNT': '503.25',
  'SL_AMOUNT': '28'},
 {'DOCNO': 'IV-00139',
  'SL_CODE': '300-2026',
  'SL_DATE': '05/03/2025',
  'PI_DOCNO': 'PI-00183',
  'PI_CODE': '2026',
  'PI_DATE': '05/03/2025',
  'PI_AMOUNT': '4712.25',
  'SL_AMOUNT': '499.8'},
 {'DOCNO': 'IV-00140',
  'SL_CODE': '300-2033',
  'SL_DATE': '03/03/2025',
  'PI_DOCNO': 'PI-00185',
  'PI_CODE': '2033',
  'PI_DATE': '03/03/2025',
  'PI_AMOUNT': '33398.65',
  'SL_AMOUNT': '1010.24'},
 {'DOCNO':

In [48]:
CONTRA = {
    'DOCNO': 'IV-00138',
    'SL_CODE': '300-1039',
    'SL_DATE': '03/03/2025',
    'PI_DOCNO': 'PI-00180',
    'PI_CODE': '1039',
    'PI_DATE': '03/03/2025',
    'PI_AMOUNT': 503.25,
    'SL_AMOUNT': 28.00
}
DocNo = 15
DocNo = f"CT-{DocNo:05d}"
# # Customer Contra
AR_CT = ComServer.BizObjects.Find("AR_CT")
AR_CT_lMain = AR_CT.DataSets.Find("MainDataSet")
AR_CT_lDetail = AR_CT.DataSets.Find("cdsKnockOff")

lDocKey = AR_CT.FindKeyByRef("DocNo", DocNo)
if lDocKey is None:
    AR_CT.New()
    AR_CT_lMain.FindField("DocNo").AsString = DocNo
    AR_CT_lMain.FindField("DocDate").value = "31/03/2025"
    AR_CT_lMain.FindField("PostDate").value = "11/04/2025"
    AR_CT_lMain.FindField("Code").AsString = CONTRA.get('SL_CODE') # "300-1039"
    AR_CT_lMain.FindField("Description").AsString = ""
    AR_CT_lMain.FindField("DocAmt").AsFloat = CONTRA.get('SL_AMOUNT') # 28.00
    
    #Knock Off IV  
    V = ["IV", CONTRA.get('DOCNO')]  #DocType, DocNo
    if (AR_CT_lDetail.Locate("DocType;DocNo", V, False, False)) :
        AR_CT_lDetail.Edit()
        AR_CT_lDetail.FindField("KOAmt").AsFloat = CONTRA.get('SL_AMOUNT')
        AR_CT_lDetail.FindField("KnockOff").AsString = "T"
        AR_CT_lDetail.Post()
else:
    raise Exception(f"Record Found, cannot create new record")

try:
    AR_CT.Save()
    print(f"Posting Customher Contra {DocNo} Done")
except Exception as e:
    print("Oops!", e)
    raise e
AR_CT.Close()

# # Supplier Contra
AP_ST = ComServer.BizObjects.Find("AP_ST")
AP_ST_lMain = AP_ST.DataSets.Find("MainDataSet")
AP_ST_lDetail = AP_ST.DataSets.Find("cdsKnockOff")

lDocKey = AP_ST.FindKeyByRef("DocNo", DocNo)
if lDocKey is None:
    AP_ST.New()
    AP_ST_lMain.FindField("DocNo").AsString = DocNo
    AP_ST_lMain.FindField("DocDate").value = "31/03/2025"
    AP_ST_lMain.FindField("PostDate").value = "11/04/2025"
    AP_ST_lMain.FindField("Code").AsString = CONTRA.get('PI_CODE') # "1039"
    AP_ST_lMain.FindField("Description").AsString = ""
    AP_ST_lMain.FindField("DocAmt").AsFloat = CONTRA.get('SL_AMOUNT') # 503.25
    
    #Knock Off IV  
    V = ["PI", CONTRA.get('PI_DOCNO')]  #DocType, DocNo
    if (AP_ST_lDetail.Locate("DocType;DocNo", V, False, False)) :
        AP_ST_lDetail.Edit()
        AP_ST_lDetail.FindField("KOAmt").AsFloat = CONTRA.get('SL_AMOUNT')
        AP_ST_lDetail.FindField("KnockOff").AsString = "T"
        AP_ST_lDetail.Post()
else:
    AP_ST.Params.Find("Dockey").Value = lDocKey
    AP_ST.Open()
    AP_ST.Edit()
    AP_ST_lMain.Edit()
    AP_ST_lMain.FindField("Code").AsString = CONTRA.get('PI_CODE') # "1039"
    AP_ST_lMain.FindField("DocDate").value = "31/03/2025"
    AP_ST_lMain.FindField("PostDate").value = "11/04/2025"
    AP_ST_lMain.FindField("DocAmt").AsFloat = CONTRA.get('SL_AMOUNT')
    AP_ST_lMain.FindField("Description").AsString = f"From Customer Contra {DocNo}"

    #Knock Off IV  
    V = ["PI", CONTRA.get('PI_DOCNO')]  #DocType, DocNo
    if (AP_ST_lDetail.Locate("DocType;DocNo", V, False, False)) :
        AP_ST_lDetail.Edit()
        AP_ST_lDetail.FindField("KOAmt").AsFloat = CONTRA.get('SL_AMOUNT')
        AP_ST_lDetail.FindField("KnockOff").AsString = "T"
        AP_ST_lDetail.Post()
try:
    AP_ST.Save()
    print(f"Posting Supplier Contra {DocNo} Done")
except Exception as e:
    print("Oops!", e)
    raise e
AP_ST.Close()



Posting Customher Contra CT-00015 Done
Posting Supplier Contra CT-00015 Done


In [42]:
lSQL = "SELECT DOCNO FROM AP_ST"
lDataSet = ComServer.DBManager.NewDataSet(lSQL)
binded = GetResult(lDataSet)
binded

[{'DOCNO': 'CT-00001'},
 {'DOCNO': 'CT-00002'},
 {'DOCNO': 'CT-00004'},
 {'DOCNO': 'CT-00005'},
 {'DOCNO': 'CT-00007'},
 {'DOCNO': 'CT-00008'}]

In [ ]:
from pydantic import BaseModel
from typing import List
import datetime

class SL_IV_Detail(BaseModel):
    Seq: int
    Account: str
    Remark1: str = ""
    ItemCode: str
    Qty: float
    UnitPrice: float
    Amount: float = 0.0
    
    def set_field(self, lDetail):
        lDetail.Append()
        lDetail.FindField("Seq").value = self.Seq
        lDetail.FindField("Account").AsString = self.Account
        lDetail.FindField("REMARK1").AsString = self.Remark1
        lDetail.FindField("ItemCode").AsString = self.ItemCode
        lDetail.FindField("Qty").AsFloat = self.Qty
        lDetail.FindField("UnitPrice").AsFloat = self.UnitPrice
        lDetail.FindField("Amount").AsFloat = self.Amount
        lDetail.Post()
        
class SL_IV(BaseModel):
    DocNo: str
    DocDate: str
    Code: str
    Description: str = ""
    
    cdsDocDetail: List[SL_IV_Detail] = []
    
    def set_field(self, lMain, post_date: str = datetime.datetime.now().strftime('%d/%m/%Y')):
        lMain.FindField("DocNo").AsString = self.DocNo
        lMain.FindField("DocDate").value = self.DocDate
        lMain.FindField("PostDate").value = post_date
        lMain.FindField("Code").AsString = self.Code
        lMain.FindField("Description").AsString = self.Description
        
    @classmethod
    def post(cls):
        BizObject = ComServer.BizObjects.Find("SL_IV")
        lMain = BizObject.DataSets.Find("MainDataSet")
        lDetail = BizObject.DataSets.Find("cdsDocDetail")
        
        lDocKey = BizObject.FindKeyByRef("DocNo", cls.DocNo)
        
        if lDocKey is None:
            BizObject.New()
            cls.set_field(lMain)
            
            for detail in cls.cdsDocDetail:
                detail.set_field(lDetail)
        else:
            raise Exception(f"Record Found, cannot create new record")
        
        try:
            BizObject.Save()
            print(f"Posting {cls.DocNo} Done")
        except Exception as e:
            print("Oops!", e)
        BizObject.Close()


    
class SL_IV_Service:
    
    @classmethod
    def post(cls, sales: SL_IV):
        BizObject = ComServer.BizObjects.Find("SL_IV")
        lMain = BizObject.DataSets.Find("MainDataSet")
        lDetail = BizObject.DataSets.Find("cdsDocDetail")
        
        lDocKey = BizObject.FindKeyByRef("DocNo", sales.DocNo)
        
        if lDocKey is None:
            BizObject.New()
            sales.set_field(lMain)
            
            for detail in sales.cdsDocDetail:
                detail.set_field(lDetail)
        else:
            raise Exception(f"Record Found, cannot create new record")
        
        try:
            BizObject.Save()
            print(f"Posting {sales.DocNo} Done")
        except Exception as e:
            print("Oops!", e)
        BizObject.Close()
        

## Example config toml

```toml
[SQLACC]
username = "USER"
password = "PASSWORD"
dfc = "\\\\path\\Default.DCF"
fdb = "DATABASE.FDB"
```

In [8]:
import win32com.client
from typing import List, Dict, Any
import os
from time import sleep
import tomllib

with open('config.toml', 'rb') as f:
    sqlacc = tomllib.load(f)['SQLACC']
    
ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
class ServerManager:
    @classmethod
    def KillApp(cls):
        os.system('cmd /c "taskkill /IM "SQLACC.exe" /F"')
        sleep(2) #sleep 2 sec
    @classmethod
    def CheckLogin(cls):
        global ComServer
        ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
        B = ComServer.IsLogin
        if B == True:
            ComServer.Logout()
            cls.KillApp()
        ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
        try:    
            ComServer.Login(sqlacc.get('username', ''), sqlacc.get('password', ''), #UserName, Password
                            sqlacc.get('dfc', ''), #DCF file
                            sqlacc.get('fdb', '')) #Database Name
        except Exception as e:
            print("Oops !", e)
            
    @classmethod
    def Get():
        return ComServer
            
class SQLUtils:
    @classmethod
    def ShowResult(cls, ADataset):
        if ADataset.RecordCount > 0:
            while not ADataset.eof:
                fc = ADataset.Fields.Count
                for x in range(fc):
                    fn = ADataset.Fields.Items(x).FieldName
                    fv = ADataset.FindField(fn).AsString
                    lresult = "Index : "+ str(x) + " FieldName : " + fn + " Value : " + fv
                    print (lresult)
                print("====")
                ADataset.Next()
        else:
            print ("Record Not Found")
            
    @classmethod
    def GetResult(cls, ADataset) -> List[Dict[str, Any]] | None:
        Result = []
        if ADataset.RecordCount > 0:
            while not ADataset.eof:
                fc = ADataset.Fields.Count
                lData = {}
                for x in range(fc):
                    fn = ADataset.Fields.Items(x).FieldName
                    fv = ADataset.FindField(fn).AsString
                    lData[fn] = fv
                Result.append(lData)
                ADataset.Next()
            return Result
        else:
            return None


# lSQL = "SELECT * FROM AR_CT A "
# lSQL += "INNER JOIN AR_KNOCKOFF B ON (A.DOCKEY = B.DOCKEY)"
lSQL = """SELECT DOCKEY, DOCNO, DOCDATE, CODE, COMPANYNAME, DOCAMT FROM SL_IV
"""
lDataSet = ComServer.DBManager.NewDataSet(lSQL)
res = SQLUtils.GetResult(lDataSet)
# import pprint
# pprint.pprint(res)

# BizObject = ComServer.BizObjects.Find('AR_CT')
# lDetailDataSet = BizObject.DataSets.Find('cdsKnockOff')
# # Select(Fields, Where, OrderBy, OutputFormat, Delimiter, OutputFile): String; 
# FileName = lDetailDataSet.Select("*", "", "", "AD", ",", "C:\\Users\\D&B Trading\\Downloads\\AR_CT.json")

import polars as pl
df = pl.DataFrame(res)
indexes = [0]
# df[indexes]
df

DOCKEY,DOCNO,DOCDATE,CODE,COMPANYNAME,DOCAMT
str,str,str,str,str,str
"""1""","""IV-00001""","""6/11/2025""","""300-T0001""","""THAI TEA TRADING""","""800"""
"""5""","""IV-00002""","""6/11/2025""","""300-W0002""","""WENDY DESIGN GROUP""","""5990"""
"""8""","""IV-00003""","""6/29/2025""","""300-H0001""","""YA SQUARE""","""14000"""
"""15""","""IV-00004""","""6/29/2025""","""300-A0001""","""A HOME FURNISHING & SERVICE""","""1370"""
"""18""","""IV-00005""","""6/29/2025""","""300-C0001""","""C ART TECHNOLOGY HUB""","""1200"""
…,…,…,…,…,…
"""44""","""IV-00013""","""7/8/2025""","""300-G0003""","""GOHILLS INTERNATIONAL GROUP""","""28700"""
"""55""","""IV-00014""","""7/8/2025""","""300-H0001""","""YA SQUARE""","""1200"""
"""57""","""IV-00015""","""7/9/2025""","""300-G0001""","""GOGO HOME DECOR""","""12900"""


In [12]:
from pydantic import BaseModel
from typing import List, Dict, Any

class QueryObject(BaseModel):
    query_obj: Dict[str, Any] = {}
    
    def select(
        self, 
        fields: str | List[str] = None
    ) -> "QueryObject":
        self.query_obj["SELECT"] = ", ".join(fields) if isinstance(fields, list) else (fields or "*")
        return self
    
    def where(
        self,
        **kwargs
    ) -> "QueryObject":
        if "WHERE" not in self.query_obj:
            self.query_obj["WHERE"] = {}
        
        self.query_obj["WHERE"].update(kwargs)
        return self
    
    def orderby(
        self,
        fields: str | List[str] = None
    ) -> "QueryObject":
        self.query_obj["ORDER BY"] = ", ".join(fields) if isinstance(fields, list) else (fields or "")
        return self
    
    def query(self) -> str:
        query = f"SELECT {self.query_obj['SELECT']} FROM {self.Metadata.tablename}"
        if "WHERE" in self.query_obj:
            query += " WHERE " + " AND ".join([f"{k} = {v}" for k, v in self.query_obj["WHERE"].items()])
        if "ORDER BY" in self.query_obj:
            query += " ORDER BY " + self.query_obj["ORDER BY"]
        return query
    
    class Metadata:
       tablename = ''


    
class CustomerContra(QueryObject):   
    DOCKEY: str = ''
    DOCNO: str = ''
    CODE: str = ''
    DOCDATE: str = ''
    POSTDATE: str = ''
    TAXDATE: str = ''
    DESCRIPTION: str = ''
    AREA: str = ''
    AGENT: str = ''
    JOURNAL: str = ''
    PROJECT: str = ''
    CURRENCYCODE: str = ''
    CURRENCYRATE: str = ''
    DOCAMT: str = ''
    LOCALDOCAMT: str = ''
    UNAPPLIEDAMT: str = ''
    GLTRANSID: str = ''
    CANCELLED: str = ''
    UPDATECOUNT: str = ''
    ATTACHMENTS: str = ''
    NOTE: str = ''
    LASTMODIFIED: str = ''
    DOCNOSETKEY: str = ''
    NEXTDOCNO: str = ''
    CHANGED: str = ''

    class Metadata:
       tablename = 'AR_CT'
    
ar_obj = CustomerContra()
ar_obj.select(['DOCKEY', 'DOCNO', 'CODE']).where(DOCKEY='1234').query()

'SELECT DOCKEY, DOCNO, CODE FROM AR_CT WHERE DOCKEY = 1234'

In [ ]:
class SalesInvoice(QueryObject):
    ...
    DOCKEY: str = ''
    DOCNO: str = ''
    DOCNOEX: str = ''
    DOCDATE: str = ''
    POSTDATE: str = ''
    TAXDATE: str = ''
    EIV_UTC: str = ''
    CODE: str = ''
    COMPANYNAME: str = ''
    ADDRESS1: str = ''

    """
    1. DOCKEY   DataType: 3   Size: 4
2. DOCNO   DataType: 24   Size: 82
3. DOCNOEX   DataType: 24   Size: 82
4. DOCDATE   DataType: 9   Size: 4
5. POSTDATE   DataType: 9   Size: 4
6. TAXDATE   DataType: 9   Size: 4
7. EIV_UTC   DataType: 36   Size: 16
8. CODE   DataType: 24   Size: 22
9. COMPANYNAME   DataType: 24   Size: 202
10. ADDRESS1   DataType: 24   Size: 122
11. ADDRESS2   DataType: 24   Size: 122
12. ADDRESS3   DataType: 24   Size: 122
13. ADDRESS4   DataType: 24   Size: 122
14. POSTCODE   DataType: 24   Size: 22
15. CITY   DataType: 24   Size: 102
16. STATE   DataType: 24   Size: 102
17. COUNTRY   DataType: 24   Size: 6
18. PHONE1   DataType: 24   Size: 402
19. MOBILE   DataType: 24   Size: 402
20. FAX1   DataType: 24   Size: 402
21. ATTENTION   DataType: 24   Size: 142
22. AREA   DataType: 24   Size: 22
23. AGENT   DataType: 24   Size: 22
24. PROJECT   DataType: 24   Size: 42
25. TERMS   DataType: 24   Size: 22
26. CURRENCYCODE   DataType: 24   Size: 14
27. CURRENCYRATE   DataType: 37   Size: 34
28. SHIPPER   DataType: 24   Size: 62
29. DESCRIPTION   DataType: 24   Size: 602
30. CANCELLED   DataType: 5   Size: 2
31. DOCAMT   DataType: 37   Size: 34
32. LOCALDOCAMT   DataType: 37   Size: 34
33. D_AMOUNT   DataType: 37   Size: 34
34. VALIDITY   DataType: 24   Size: 602
35. DELIVERYTERM   DataType: 24   Size: 602
36. CC   DataType: 24   Size: 602
37. DOCREF1   DataType: 24   Size: 82
38. DOCREF2   DataType: 24   Size: 82
39. DOCREF3   DataType: 24   Size: 82
40. DOCREF4   DataType: 24   Size: 82
41. BRANCHNAME   DataType: 24   Size: 202
42. DADDRESS1   DataType: 24   Size: 122
43. DADDRESS2   DataType: 24   Size: 122
44. DADDRESS3   DataType: 24   Size: 122
45. DADDRESS4   DataType: 24   Size: 122
46. DPOSTCODE   DataType: 24   Size: 22
47. DCITY   DataType: 24   Size: 102
48. DSTATE   DataType: 24   Size: 102
49. DCOUNTRY   DataType: 24   Size: 6
50. DATTENTION   DataType: 24   Size: 142
51. DPHONE1   DataType: 24   Size: 402
52. DMOBILE   DataType: 24   Size: 402
53. DFAX1   DataType: 24   Size: 402
54. TAXEXEMPTNO   DataType: 24   Size: 102
55. SALESTAXNO   DataType: 24   Size: 52
56. SERVICETAXNO   DataType: 24   Size: 52
57. TIN   DataType: 24   Size: 30
58. IDTYPE   DataType: 2   Size: 2
59. IDNO   DataType: 24   Size: 42
60. TOURISMNO   DataType: 24   Size: 36
61. SIC   DataType: 24   Size: 22
62. INCOTERMS   DataType: 24   Size: 8
63. SUBMISSIONTYPE   DataType: 3   Size: 4
64. IRBM_STATUS   DataType: 3   Size: 4
65. IRBM_INTERNALID   DataType: 24   Size: 82
66. IRBM_UUID   DataType: 1   Size: 27
67. IRBM_LONGID   DataType: 1   Size: 51
68. EIVRequest_UUID   DataType: 1   Size: 37
69. ATTACHMENTS   DataType: 15   Size: 0
70. NOTE   DataType: 15   Size: 0
71. UPDATECOUNT   DataType: 3   Size: 4
72. TRANSFERABLE   DataType: 5   Size: 2
73. PRINTCOUNT   DataType: 3   Size: 4
74. LASTMODIFIED   DataType: 25   Size: 8
75. sdsDocDetail   DataType: 29   Size: 0
76. CHANGED   DataType: 5   Size: 2
77. DocNoSetKey   DataType: 25   Size: 8
78. NextDocNo   DataType: 24   Size: 82

<?xml version="1.0" standalone="yes"?>
<DATAPACKET Version="2.0">
  <METADATA>
    <FIELDS>
      <FIELD attrname="DOCKEY" fieldtype="i4" required="true"/>
      <FIELD attrname="DOCNO" fieldtype="string.uni" required="true" SUBTYPE="FixedChar" WIDTH="80"/>
      <FIELD attrname="DOCNOEX" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="80"/>
      <FIELD attrname="DOCDATE" fieldtype="date"/>
      <FIELD attrname="POSTDATE" fieldtype="date"/>
      <FIELD attrname="TAXDATE" fieldtype="date"/>
      <FIELD attrname="EIV_UTC" fieldtype="SQLdateTime" SUBTYPE="Formatted"/>
      <FIELD attrname="CODE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
      <FIELD attrname="COMPANYNAME" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="200"/>
      <FIELD attrname="ADDRESS1" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="120"/>
      <FIELD attrname="ADDRESS2" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="120"/>
      <FIELD attrname="ADDRESS3" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="120"/>
      <FIELD attrname="ADDRESS4" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="120"/>
      <FIELD attrname="POSTCODE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
      <FIELD attrname="CITY" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="100"/>
      <FIELD attrname="STATE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="100"/>
      <FIELD attrname="COUNTRY" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="4"/>
      <FIELD attrname="PHONE1" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
      <FIELD attrname="MOBILE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
      <FIELD attrname="FAX1" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
      <FIELD attrname="ATTENTION" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="140"/>
      <FIELD attrname="AREA" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
      <FIELD attrname="AGENT" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
      <FIELD attrname="PROJECT" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="40"/>
      <FIELD attrname="TERMS" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
      <FIELD attrname="CURRENCYCODE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="12"/>
      <FIELD attrname="CURRENCYRATE" fieldtype="fixedFMT" DECIMALS="8" WIDTH="18"/>
      <FIELD attrname="SHIPPER" fieldtype="string.uni" required="true" SUBTYPE="FixedChar" WIDTH="60"/>
      <FIELD attrname="DESCRIPTION" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="600"/>
      <FIELD attrname="CANCELLED" fieldtype="boolean"/>
      <FIELD attrname="DOCAMT" fieldtype="fixedFMT" DECIMALS="2" WIDTH="18"/>
      <FIELD attrname="LOCALDOCAMT" fieldtype="fixedFMT" DECIMALS="2" WIDTH="18"/>
      <FIELD attrname="D_AMOUNT" fieldtype="fixedFMT" DECIMALS="2" WIDTH="18"/>
      <FIELD attrname="VALIDITY" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="600"/>
      <FIELD attrname="DELIVERYTERM" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="600"/>
      <FIELD attrname="CC" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="600"/>
      <FIELD attrname="DOCREF1" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="80"/>
      <FIELD attrname="DOCREF2" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="80"/>
      <FIELD attrname="DOCREF3" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="80"/>
      <FIELD attrname="DOCREF4" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="80"/>
      <FIELD attrname="BRANCHNAME" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="200"/>
      <FIELD attrname="DADDRESS1" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="120"/>
      <FIELD attrname="DADDRESS2" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="120"/>
      <FIELD attrname="DADDRESS3" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="120"/>
      <FIELD attrname="DADDRESS4" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="120"/>
      <FIELD attrname="DPOSTCODE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
      <FIELD attrname="DCITY" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="100"/>
      <FIELD attrname="DSTATE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="100"/>
      <FIELD attrname="DCOUNTRY" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="4"/>
      <FIELD attrname="DATTENTION" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="140"/>
      <FIELD attrname="DPHONE1" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
      <FIELD attrname="DMOBILE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
      <FIELD attrname="DFAX1" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
      <FIELD attrname="TAXEXEMPTNO" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="100"/>
      <FIELD attrname="SALESTAXNO" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="50"/>
      <FIELD attrname="SERVICETAXNO" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="50"/>
      <FIELD attrname="TIN" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="28"/>
      <FIELD attrname="IDTYPE" fieldtype="i2"/>
      <FIELD attrname="IDNO" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="40"/>
      <FIELD attrname="TOURISMNO" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="34"/>
      <FIELD attrname="SIC" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
      <FIELD attrname="INCOTERMS" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="6"/>
      <FIELD attrname="SUBMISSIONTYPE" fieldtype="i4"/>
      <FIELD attrname="IRBM_STATUS" fieldtype="i4"/>
      <FIELD attrname="IRBM_INTERNALID" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="80"/>
      <FIELD attrname="IRBM_UUID" fieldtype="string" SUBTYPE="FixedChar" WIDTH="26"/>
      <FIELD attrname="IRBM_LONGID" fieldtype="string" SUBTYPE="FixedChar" WIDTH="50"/>
      <FIELD attrname="EIVREQUEST_UUID" fieldtype="string" SUBTYPE="FixedChar" WIDTH="36"/>
      <FIELD attrname="ATTACHMENTS" fieldtype="bin.hex" SUBTYPE="Binary" WIDTH="8"/>
      <FIELD attrname="NOTE" fieldtype="bin.hex" SUBTYPE="Binary" WIDTH="8"/>
      <FIELD attrname="TRANSFERABLE" fieldtype="boolean"/>
      <FIELD attrname="UPDATECOUNT" fieldtype="i4"/>
      <FIELD attrname="PRINTCOUNT" fieldtype="i4"/>
      <FIELD attrname="LASTMODIFIED" fieldtype="i8"/>
      <FIELD attrname="DOCNOSETKEY" fieldtype="i8" required="true"/>
      <FIELD attrname="NEXTDOCNO" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="80"/>
      <FIELD attrname="CHANGED" fieldtype="boolean" required="true"/>
      <FIELD attrname="sdsDocDetail" fieldtype="nested">
        <FIELDS>
          <FIELD attrname="DTLKEY" fieldtype="i4" required="true"/>
          <FIELD attrname="DOCKEY" fieldtype="i4" required="true"/>
          <FIELD attrname="SEQ" fieldtype="i4"/>
          <FIELD attrname="STYLEID" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="10"/>
          <FIELD attrname="NUMBER" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="10"/>
          <FIELD attrname="ITEMCODE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="60"/>
          <FIELD attrname="LOCATION" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="40"/>
          <FIELD attrname="BATCH" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="60"/>
          <FIELD attrname="PROJECT" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="40"/>
          <FIELD attrname="DESCRIPTION" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
          <FIELD attrname="DESCRIPTION2" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
          <FIELD attrname="DESCRIPTION3" fieldtype="bin.hex" SUBTYPE="Binary" WIDTH="8"/>
          <FIELD attrname="PERMITNO" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="40"/>
          <FIELD attrname="QTY" fieldtype="fixedFMT" DECIMALS="4" WIDTH="18"/>
          <FIELD attrname="UOM" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
          <FIELD attrname="RATE" fieldtype="fixedFMT" DECIMALS="4" WIDTH="18"/>
          <FIELD attrname="SQTY" fieldtype="fixedFMT" DECIMALS="4" WIDTH="18"/>
          <FIELD attrname="SUOMQTY" fieldtype="fixedFMT" DECIMALS="4" WIDTH="18"/>
          <FIELD attrname="UNITPRICE" fieldtype="fixedFMT" DECIMALS="8" WIDTH="18"/>
          <FIELD attrname="DELIVERYDATE" fieldtype="date"/>
          <FIELD attrname="DISC" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="40"/>
          <FIELD attrname="TAX" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
          <FIELD attrname="TARIFF" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="40"/>
          <FIELD attrname="TAXEXEMPTIONREASON" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="600"/>
          <FIELD attrname="IRBM_CLASSIFICATION" fieldtype="string" SUBTYPE="FixedChar" WIDTH="3"/>
          <FIELD attrname="TAXRATE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="40"/>
          <FIELD attrname="TAXAMT" fieldtype="fixedFMT" DECIMALS="2" WIDTH="18"/>
          <FIELD attrname="LOCALTAXAMT" fieldtype="fixedFMT" DECIMALS="2" WIDTH="18"/>
          <FIELD attrname="TAXINCLUSIVE" fieldtype="boolean"/>
          <FIELD attrname="AMOUNT" fieldtype="fixedFMT" DECIMALS="2" WIDTH="18"/>
          <FIELD attrname="LOCALAMOUNT" fieldtype="fixedFMT" DECIMALS="2" WIDTH="18"/>
          <FIELD attrname="TAXABLEAMT" fieldtype="fixedFMT" DECIMALS="2" WIDTH="18"/>
          <FIELD attrname="ACCOUNT" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="20"/>
          <FIELD attrname="PRINTABLE" fieldtype="boolean"/>
          <FIELD attrname="FROMDOCTYPE" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="4"/>
          <FIELD attrname="FROMDOCKEY" fieldtype="i4"/>
          <FIELD attrname="FROMDTLKEY" fieldtype="i4"/>
          <FIELD attrname="TRANSFERABLE" fieldtype="boolean"/>
          <FIELD attrname="REMARK1" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
          <FIELD attrname="REMARK2" fieldtype="string.uni" SUBTYPE="FixedChar" WIDTH="400"/>
          <FIELD attrname="INITIALPURCHASECOST" fieldtype="fixedFMT" DECIMALS="2" WIDTH="18"/>
          <FIELD attrname="CHANGED" fieldtype="boolean" required="true"/>
          <FIELD attrname="CompanyItemCode" fieldtype="string.uni" WIDTH="60"/>
          <FIELD attrname="sdsSerialNumber" fieldtype="nested">
            <FIELDS>
              <FIELD attrname="SERIALNUMBER" fieldtype="string.uni" required="true" SUBTYPE="FixedChar" WIDTH="100"/>
            </FIELDS>
            <PARAMS/>
          </FIELD>
        </FIELDS>
        <PARAMS/>
      </FIELD>
    </FIELDS>
    <PARAMS/>
  </METADATA>
  <ROWDATA/>
</DATAPACKET>

    """

In [ ]:
from services.sqlacc import initialize_com, SQLUtils

# BizObject = initialize_com().BizObjects.Find("AR_CT")
# lMain = BizObject.DataSets.Find("MainDataSet")
# lDetail = BizObject.DataSets.Find("cdsKnockOff")
# lSQL = "SELECT A.*, B.NEXTNUMBER FROM SY_DOCNO A "
# lSQL = lSQL + "INNER JOIN SY_DOCNO_DTL B ON (A.DOCKEY=B.PARENTKEY) "
# lSQL = lSQL + "WHERE A.DOCTYPE='CT' "
# # lSQL = lSQL + "AND A.DESCRIPTION='Customer Invoice' "
# # lSQL = lSQL + "AND A.STATESET=1 "

lSQL = """SELECT A.*, B.NEXTNUMBER FROM SY_DOCNO A 
INNER JOIN SY_DOCNO_DTL B ON (A.DOCKEY=B.PARENTKEY)
WHERE A.DOCTYPE='KO'
;
"""
lSQL = "SELECT * FROM SY_DOCNO"
lDataSet = initialize_com().DBManager.NewDataSet(lSQL)
# NextNo = lDataSet.FindField('NEXTNUMBER').AsFloat
# NextNo = int(NextNo)
# f"CT-{NextNo:0>5}"
res = SQLUtils.GetResult(lDataSet)
import json
with open('SY_DOCNO.json', 'w') as f:
    json.dump(res, f)

In [71]:
import datetime


def get_start_index():
    lSQL = """SELECT A.*, B.NEXTNUMBER FROM SY_DOCNO A 
    INNER JOIN SY_DOCNO_DTL B ON (A.DOCKEY=B.PARENTKEY)
    WHERE A.DOCTYPE='CT'
    ;
    """
    lDataSet = initialize_com().DBManager.NewDataSet(lSQL)
    NextNo = lDataSet.FindField('NEXTNUMBER').AsFloat
    NextNo = int(NextNo)
    return NextNo

today = datetime.date(2025, 6, 30)
first_day = today.replace(day=1)

lSQL = "SELECT DOCKEY, DOCNO, DOCDATE, CODE, COMPANYNAME, DOCAMT FROM SL_IV"
lSQL += f" WHERE DOCDATE BETWEEN '{first_day}' AND '{today}'"
ComServer = initialize_com()
lDataSet = ComServer.DBManager.NewDataSet(lSQL)
res = SQLUtils.GetResult(lDataSet)
df = pl.DataFrame(res)
# df

start_index = get_start_index()
data = []
for row in df[[3]].iter_rows(named=True):
    # dSQL = f"""SELECT * FROM SL_IVDTL
    # WHERE DOCKEY = {row.get("DOCKEY")}
    # """
    temp = {
        "DOCNO": f"CT-{start_index:0>5}",
        "DOCDATE": datetime.datetime.now(), #TODO: Get from input
        "CODE": row.get("CODE"),
        "DOCAMT": row.get("DOCAMT"),
        "LOCALDOCAMT": row.get("DOCAMT"),
        "KOCKOFF": [
            {
                "DOCTYPE": "IV",
                "DOCKEY": row.get("DOCKEY"),
                "DOCNO": row.get("DOCNO"),
                "DOCAMT": row.get("DOCAMT"),
                "LOCALDOCAMT": row.get("DOCAMT")
            }
        ]
    }
    data.append(temp)
    start_index += 1



def post_contra(data: dict[str, Any]):
    BizObject = initialize_com().BizObjects.Find("AR_CT")
    lMain = BizObject.DataSets.Find("MainDataSet")
    lDetail = BizObject.DataSets.Find("cdsKnockOff")

    lSQL = """SELECT A.*, B.NEXTNUMBER FROM SY_DOCNO A 
    INNER JOIN SY_DOCNO_DTL B ON (A.DOCKEY=B.PARENTKEY)
    WHERE A.DOCTYPE='CT'
    ;
    """
    lDataSet = initialize_com().DBManager.NewDataSet(lSQL)
    NextNo = lDataSet.FindField('NEXTNUMBER').AsFloat
    NextNo = int(NextNo)
    DOCNO=f"CT-{NextNo:0>5}"


    # set main
    lDate = datetime.datetime(2023, 2, 22, 13, 0)
    lDate.strftime('%m/%d/%Y')
    BizObject.New()
    lMain.FindField("DocNo").AsString = DOCNO
    lMain.FindField("DocDate").value = datetime.datetime.now()
    lMain.FindField("Code").AsString = data.get("CODE")   #"300-C0001" #Customer Account
    lMain.FindField("DocAmt").AsFloat = data.get("DOCAMT")
    lMain.FindField("LocalDocAmt").AsFloat = data.get("LOCALDOCAMT")

    # lDetail.Append()
    # lDetail.FindField("
    for item in data.get("KOCKOFF"):
        lDetail.Append()
        print(item)
        lDetail.FindField("DocType").AsString = item.get("DOCTYPE")
        lDetail.FindField("DocNo").AsString = item.get("DOCNO")
        lDetail.FindField("Project").AsString = "----"
        lDetail.FindField("DocAmt").AsFloat = float(item.get("DOCAMT"))
        lDetail.FindField("KOAmt").AsFloat = 0
        lDetail.FindField("DocDate").value = datetime.datetime.now()

        lDetail.Post()

    

    BizObject.Save()
    BizObject.Close()


for row in data:
    post_contra(row)

{'DOCTYPE': 'IV', 'DOCKEY': '15', 'DOCNO': 'IV-00004', 'DOCAMT': '1370', 'LOCALDOCAMT': '1370'}


com_error: (-2147352567, 'Exception occurred.', (0, None, 'Field value required.', None, 0, -2147418113), None)

In [ ]:
from services.sqlacc import initialize_com, SQLUtils

lSQL = """SELECT A.*, B.COMPANYNAME, (A.DOCAMT-A.PAYMENTAMT) OUTSTANDING  FROM AR_IV A
	         INNER JOIN AR_CUSTOMER B ON (A.CODE=B.CODE) 
			 WHERE (A.DOCAMT-A.PAYMENTAMT)<>0
			 AND CANCELLED='False'
			 """

res = initialize_com().DBManager.NewDataSet(lSQL)
res = SQLUtils.GetResult(res)
import polars as pl
df = pl.DataFrame(res)
df

DOCKEY,DOCNO,DOCNOEX,CODE,JOURNAL,DOCDATE,POSTDATE,TAXDATE,TERMS,DUEDATE,DESCRIPTION,AREA,AGENT,PROJECT,CURRENCYCODE,CURRENCYRATE,DOCAMT,LOCALDOCAMT,PAYMENTAMT,FROMDOCTYPE,TAXEXEMPTNO,GLTRANSID,CANCELLED,UPDATECOUNT,ATTACHMENTS,NOTE,LASTMODIFIED,COMPANYNAME,OUTSTANDING
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""15""","""IV-00004""","""""","""300-A0001""","""SALES""","""6/29/2025""","""6/29/2025""","""6/29/2025""","""30 Days""","""7/29/2025""","""Sales""","""KL""","""Yuki""","""----""","""----""","""1""","""1370""","""1370""","""0""","""IV""","""""","""5""","""False""","""""","""""","""""","""1736148323""","""A HOME FURNISHING & SERVICE""","""1370"""
"""20""","""IV-00006""","""""","""300-G0002""","""SALES""","""6/29/2025""","""6/29/2025""","""6/29/2025""","""30 Days""","""7/29/2025""","""Sales""","""KL""","""Gabriel""","""----""","""----""","""1""","""2470""","""2470""","""0""","""IV""","""""","""7""","""False""","""""","""""","""""","""1736148324""","""G DELUXE HOTEL""","""2470"""
"""24""","""IV-00007""","""""","""300-W0001""","""SALES""","""6/30/2025""","""6/30/2025""","""6/30/2025""","""30 Days""","""7/30/2025""","""Sales""","""Selangor""","""Gabriel""","""----""","""----""","""1""","""2070""","""2070""","""130""","""IV""","""""","""8""","""False""","""""","""""","""""","""1736148325""","""WOODY FURNITURE ENTERPRISE""","""1940"""
"""31""","""IV-00008""","""""","""300-H0001""","""SALES""","""6/30/2025""","""6/30/2025""","""6/30/2025""","""30 Days""","""7/30/2025""","""Sales""","""US""","""Ashley""","""----""","""USD""","""4.05""","""1250""","""5062.5""","""0""","""IV""","""""","""9""","""False""","""""","""""","""""","""1736148325""","""YA SQUARE""","""1250"""
"""35""","""IV-00010""","""""","""300-W0002""","""SALES""","""7/1/2025""","""7/1/2025""","""7/1/2025""","""30 Days""","""7/31/2025""","""Sales""","""Selangor""","""Ashley""","""----""","""----""","""1""","""400""","""400""","""0""","""IV""","""""","""11""","""False""","""""","""""","""""","""1736148325""","""WENDY DESIGN GROUP""","""400"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""42""","""IV-00012""","""""","""300-D0001""","""SALES""","""7/7/2025""","""7/7/2025""","""7/7/2025""","""30 Days""","""8/6/2025""","""Sales""","""KL""","""Ashley""","""----""","""----""","""1""","""1900""","""1900""","""100""","""IV""","""""","""13""","""False""","""""","""""","""""","""1736148326""","""DREAM VILLA FURNITURE""","""1800"""
"""55""","""IV-00014""","""""","""300-H0001""","""SALES""","""7/8/2025""","""7/8/2025""","""7/8/2025""","""30 Days""","""8/7/2025""","""Sales""","""US""","""Ashley""","""----""","""USD""","""4.1""","""1200""","""4920""","""0""","""IV""","""""","""15""","""False""","""""","""""","""""","""1736148327""","""YA SQUARE""","""1200"""
"""57""","""IV-00015""","""""","""300-G0001""","""SALES""","""7/9/2025""","""7/9/2025""","""7/9/2025""","""30 Days""","""8/8/2025""","""Sales""","""KL""","""Ashley""","""----""","""----""","""1""","""12900""","""12900""","""0""","""IV""","""""","""16""","""False""","""""","""""","""""","""1736148328""","""GOGO HOME DECOR""","""12900"""
